In [1]:
nc = 0
mode = 1
equation = {}
constraints = {}
slack = {}
row = []
temp = []
current = ''
tableau = dict()

In [2]:
import math
import numpy as np
import copy
import pprint
intersection = ['', current]

def update_tableau():
    
    global intersection
    global count
    global current
    tableau['S'][intersection[1]] = copy.deepcopy(tableau['S'][intersection[0]])
    temp = {}
    temp = copy.deepcopy(tableau['S'])
    
    #updating S columns
    for i in tableau['S'].keys():
        for j in tableau['C'].keys():
            if i == intersection[1]:
                tableau['S'][i][j] = (np.divide(temp[intersection[0]][j], temp[intersection[0]][intersection[1]]))
            elif i == intersection[0]:
                pass
            else:
                tableau['S'][i][j] -= np.divide(np.multiply(temp[i][intersection[1]], temp[intersection[0]][j]), temp[intersection[0]][intersection[1]])
    
    #updating CB
    tableau['CB'][intersection[1]] = tableau['C'][intersection[1]]
    del tableau['CB'][intersection[0]]
        
    #updating solution columns
    tableau['solution'][intersection[1]] = tableau['solution'][intersection[0]]
    temp2 = copy.deepcopy(tableau['solution'])
    
    for i in tableau['solution']:
        if i == intersection[1]:
            tableau['solution'][i] = (np.divide(tableau['solution'][i], temp[intersection[0]][intersection[1]]))
        elif i == intersection[0]:
            pass
        else:
            #print(temp[i][intersection[1]], temp[intersection[0]][j], temp[intersection[0]][intersection[1]])
            tableau['solution'][i] -= np.divide(np.multiply(temp[i][intersection[1]], temp2[intersection[0]]), temp[intersection[0]][intersection[1]])
    
    del tableau['S'][intersection[0]]
    del tableau['solution'][intersection[0]]
    
    # Calculating Z and C-Z
    for i in tableau['C'].keys():
        tableau['Z'][i] = Z_i(i)
        tableau['C-Z'][i] = tableau['C'][i] - tableau['Z'][i]
    
    (count, max, current) = (0, -math.inf, '')

    
    for i in tableau['C-Z']:
        if tableau['C-Z'][i] > 0:
            count += 1
        if tableau['C-Z'][i] > max:
            max = tableau['C-Z'][i]
            current = i
            
    if (mode and count) or (not mode and not count):
        pass
    else:
        return
    
    temp = math.inf
    
    tableau['ratio'][intersection[1]] = copy.deepcopy(tableau['ratio'][intersection[0]])
    del tableau['ratio'][intersection[0]]
    
    intersection = ['', current]
    
    #print(tableau['ratio'], current)
    
    for i in tableau['ratio']:
        tableau['ratio'][i] = tableau['solution'][i]/tableau['S'][i][current]
        if tableau['ratio'][i] < temp:
            temp = tableau['ratio'][i]
            intersection[0] = i
    #print(intersection)

def simplex():
    
    global intersection
    while (mode and count) or (not mode and not count):
        update_tableau()
        pprint.PrettyPrinter(indent=4).pprint(tableau)
        #print(tableau['S'])
        print('\n\n')
    else:
        print('Finished Computing.....')

In [3]:
def get_data():
    global mode, nc, contraints, slack, equation
    temp = list(map(int, input('Enter the coefficients of the objective function : ').split()))
    for i in range(len(temp)):
        equation['X'+str(i+1)] = temp[i]
    mode = int(input('Enter 0 for Minimization and 1 for Maximization '))
    nc = int(input('Enter the no. of constraints '))
    print('Enter the contraints in the format C1 C2 ... Right Hand Value ');
    for i in range(nc):
        constraints['S'+str(i+1)] = list(map(int, input().split()))
        slack['S'+str(i+1)] = 0
    print(constraints, equation, slack)

In [4]:
def Z_i(key):
    sum = 0
    for i in tableau['S'].keys():
        sum += tableau['CB'][i] * tableau['S'][i][key]
    return sum

In [5]:
#Initializing Tableau
def init_tableau():

    global tableau, current, max, slack, equation, count, intersection
    (tableau['C'], tableau['Z'], tableau['C-Z'], tableau['S'], tableau['CB'], tableau['solution'], tableau['ratio']) = ({}, {}, {}, {},slack.copy(), {}, {})

    for i in constraints.keys():
        tableau['solution'][i] = constraints[i][-1]
        tableau['ratio'][i] = 0
        tableau['S'][i] = {}

    tableau['C'] = copy.deepcopy(equation)
    tableau['C'].update(slack)

    count = 1
    for i in tableau['S'].keys():
        k = 0
        for j in equation.keys():
            tableau['S'][i][j] = constraints[i][k]
            k += 1
        tableau['S'][i].update(dict([(j, ('S'+str(count) == j)*1) for j in tableau['S'].keys()]))
        count += 1

    # Calculating Z and C-Z
    for i in tableau['C'].keys():
        tableau['Z'][i] = Z_i(i)
        tableau['C-Z'][i] = tableau['C'][i] - tableau['Z'][i]

    (count, max, current) = (0, 0, '')

    for i in tableau['C-Z']:
        if tableau['C-Z'][i] > 0:
            count += 1
        if tableau['C-Z'][i] > max:
            max = tableau['C-Z'][i]
            current = i

    temp = math.inf

    intersection = ['', current]

    for i in tableau['ratio']:
        tableau['ratio'][i] = tableau['solution'][i]/tableau['S'][i][current]
        if tableau['ratio'][i] < temp:
            temp = tableau['ratio'][i]
            intersection[0] = i

    print('\n\n')
    print(tableau)
    print('\n\n')

In [6]:
def init_variables():
    global nc, mode, equation, constraints, slack, row, temp, current, tableau
    nc = 0
    mode = 1
    equation = {}
    constraints = {}
    slack = {}
    row = []
    temp = []
    current = ''
    tableau = dict()

In [7]:
if __name__ == '__main__':
    init_variables()
    get_data()
    init_tableau()
    simplex()
    result = 0
    for i in tableau['solution'].items():
        result += i[1] * tableau['CB'][i[0]]
    print("Final Result Is :", result)

Enter the coefficients of the objective function : 12 16
Enter 0 for Minimization and 1 for Maximization 1
Enter the no. of constraints 2
Enter the contraints in the format C1 C2 ... Right Hand Value 
10 20 120
8 8 80
{'S1': [10, 20, 120], 'S2': [8, 8, 80]} {'X1': 12, 'X2': 16} {'S1': 0, 'S2': 0}



{'C': {'X1': 12, 'X2': 16, 'S1': 0, 'S2': 0}, 'Z': {'X1': 0, 'X2': 0, 'S1': 0, 'S2': 0}, 'C-Z': {'X1': 12, 'X2': 16, 'S1': 0, 'S2': 0}, 'S': {'S1': {'X1': 10, 'X2': 20, 'S1': 1, 'S2': 0}, 'S2': {'X1': 8, 'X2': 8, 'S1': 0, 'S2': 1}}, 'CB': {'S1': 0, 'S2': 0}, 'solution': {'S1': 120, 'S2': 80}, 'ratio': {'S1': 6.0, 'S2': 10.0}}



{   'C': {'S1': 0, 'S2': 0, 'X1': 12, 'X2': 16},
    'C-Z': {'S1': -0.8, 'S2': 0.0, 'X1': 4.0, 'X2': 0.0},
    'CB': {'S2': 0, 'X2': 16},
    'S': {   'S2': {'S1': -0.4, 'S2': 1.0, 'X1': 4.0, 'X2': 0.0},
             'X2': {'S1': 0.05, 'S2': 0.0, 'X1': 0.5, 'X2': 1.0}},
    'Z': {'S1': 0.8, 'S2': 0.0, 'X1': 8.0, 'X2': 16.0},
    'ratio': {'S2': 8.0, 'X2': 12.0},
   